<a href="https://colab.research.google.com/github/siting1206/AIcup_NLP/blob/main/2022_AIcup_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## import package

In [1]:
!pip install torch
!pip install pandas nltk
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 32.8 MB/s 
     |████████████████████████████████| 182 kB 71.4 MB/s 
     |████████████████████████████████| 7.6 MB 51.5 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
import math
import numpy as np
import time
import torch, pandas as pd
import nltk
import re
nltk.download('punkt')

from transformers import set_seed
set_seed(123)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


cuda:0


In [4]:
# Training data file
directory="/content/drive/MyDrive/AICUP_NLP/"


file=directory+"Batch_answers - train_data (no-blank).csv"
df=pd.read_csv(file, encoding = "ISO-8859-1")

In [5]:
df = df.drop(columns=['Unnamed: 6', 'total no.: 7987'])

In [6]:
df.head()

,id,q,r,s,q',r'
0,8,"""It can go both ways . We all doubt . It is wh...","""True .""",AGREE,"""It can go both ways . We all doubt . It is wh...","""True ."""
1,8,"""It can go both ways . We all doubt . It is wh...","""True .""",AGREE,"""can go both ways . We all doubt . It is what ...","""True"""
2,8,"""It can go both ways . We all doubt . It is wh...","""True .""",AGREE,"""It can go both ways . We all doubt . It is wh...","""True"""
3,9,"""once again , you seem to support the killing ...","""based on the idea that people are dispensible...",AGREE,"""seem to support the killing of certain people""","""based on the idea that people are dispensible..."
4,9,"""once again , you seem to support the killing ...","""based on the idea that people are dispensible...",AGREE,"""you seem to support the killing of certain pe...","""based on the idea that people are dispensible"""


## Data process

In [7]:
df[['q','r',"q'","r'"]] = df[['q','r',"q'","r'"]].apply(lambda x: x.str.strip('\"'))

In [8]:
df['r'] = df['s'] + ':' + df['r']

In [9]:
df['sub_q_true'] = [1 if x in y else 0 for x,y in zip(df["q'"],df["q"])]
df['sub_r_true'] = [1 if x in y else 0 for x,y in zip(df["r'"],df["r"])]
df['sub_both'] = df['sub_q_true']*df['sub_r_true']

In [10]:
df.head()

,id,q,r,s,q',r',sub_q_true,sub_r_true,sub_both
0,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,It can go both ways . We all doubt . It is wha...,True .,1,1,1
1,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,can go both ways . We all doubt . It is what y...,True,1,1,1
2,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,It can go both ways . We all doubt . It is wha...,True,1,1,1
3,9,"once again , you seem to support the killing o...",AGREE:based on the idea that people are dispen...,AGREE,seem to support the killing of certain people,based on the idea that people are dispensible ...,1,1,1
4,9,"once again , you seem to support the killing o...",AGREE:based on the idea that people are dispen...,AGREE,you seem to support the killing of certain peo...,based on the idea that people are dispensible,1,1,1


In [11]:
data = df.loc[df['sub_both'] == 1]

In [12]:
data['q_start'] = [y.index(x) for x,y in zip(data["q'"],data["q"])]
data['r_start'] = [y.index(x) for x,y in zip(data["r'"],data["r"])]
data['q_end'] = [x+len(y)-1 for x,y in zip(data["q_start"],data["q'"])]
data['r_end'] = [x+len(y)-1 for x,y in zip(data["r_start"],data["r'"])]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [13]:
data.head()

,id,q,r,s,q',r',sub_q_true,sub_r_true,sub_both,q_start,r_start,q_end,r_end
0,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,It can go both ways . We all doubt . It is wha...,True .,1,1,1,0,6,76,11
1,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,can go both ways . We all doubt . It is what y...,True,1,1,1,3,6,74,9
2,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,It can go both ways . We all doubt . It is wha...,True,1,1,1,0,6,76,9
3,9,"once again , you seem to support the killing o...",AGREE:based on the idea that people are dispen...,AGREE,seem to support the killing of certain people,based on the idea that people are dispensible ...,1,1,1,17,6,61,98
4,9,"once again , you seem to support the killing o...",AGREE:based on the idea that people are dispen...,AGREE,you seem to support the killing of certain peo...,based on the idea that people are dispensible,1,1,1,13,6,81,50


In [14]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(data, test_size=0.2)
valid, test = train_test_split(valid, test_size=0.5)

## Tokenizer

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-large-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [17]:
train_data_q = train['q'].tolist()
valid_data_q = valid['q'].tolist()
test_data_q = test['q'].tolist()

train_data_r = train['r'].tolist()
valid_data_r = valid['r'].tolist()
test_data_r = test['r'].tolist()

In [18]:
train_encodings = tokenizer(train_data_q, train_data_r, truncation=True, padding=True)
val_encodings = tokenizer(valid_data_q, valid_data_r, truncation=True, padding=True)
test_encodings = tokenizer(test_data_q, test_data_r, truncation=True, padding=True)

In [19]:
tokenizer.decode(train_encodings['input_ids'][0])

'[CLS] As I have pointed out, the only BS here is your accusations. [SEP] DISAGREE : Not the way I see it from my experiences. I let the readers decide on their own. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [20]:
train_answer = train[['q_start', 'r_start',	'q_end', 'r_end']].to_dict('records')
valid_answer = valid[['q_start', 'r_start',	'q_end', 'r_end']].to_dict('records')
test_answer = test[['q_start', 'r_start',	'q_end', 'r_end']].to_dict('records')

In [21]:
def add_token_positions(encodings, answers):
    q_start, r_start, q_end, r_end = [],[],[],[]

    for i in range(len(answers)):
        q_start.append(encodings.char_to_token(i, answers[i]['q_start'], 0))
        r_start.append(encodings.char_to_token(i, answers[i]['r_start'], 1))
        q_end.append(encodings.char_to_token(i, answers[i]['q_end'], 0))
        r_end.append(encodings.char_to_token(i, answers[i]['r_end'], 1))

        if q_start[-1] is None:
            q_start[-1] = 0
            q_end[-1] = 0
            # continue

        if r_start[-1] is None:
            r_start[-1] = 0
            r_end[-1] = 0
            # continue

        shift = 1
        while q_end[-1] is None:
            q_end[-1] = encodings.char_to_token(i, answers[i]['q_end'] - shift)
            shift += 1
        shift = 1
        while r_end[-1] is None:
            r_end[-1] = encodings.char_to_token(i, answers[i]['r_end'] - shift)
            shift += 1
    encodings.update({'q_start':q_start, 'r_start':r_start,	'q_end':q_end, 'r_end':r_end})

In [22]:
# Convert char_based_id to token_based_id
# Find the corossponding token id after input being tokenized
add_token_positions(train_encodings, train_answer)
add_token_positions(val_encodings, valid_answer)
add_token_positions(test_encodings, test_answer)

In [23]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'q_start', 'r_start', 'q_end', 'r_end'])

In [24]:
print(train_encodings['q_start'][0])
print(train_encodings['r_start'][0])
print(train_encodings['q_end'][0])
print(train_encodings['r_end'][0])

1
22
14
31


## Dataset

In [25]:


class qrDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [26]:
train_dataset = qrDataset(train_encodings)
val_dataset = qrDataset(val_encodings)
test_dataset = qrDataset(test_encodings)

In [27]:
next(iter(train_dataset))

{'input_ids': tensor([  101,  1249,   146,  1138,  3356,  1149,   117,  1103,  1178, 21948,
          1303,  1110,  1240, 14144,   119,   102,   141,  6258, 22689, 16941,
          2036,   131,  1753,  1103,  1236,   146,  1267,  1122,  1121,  1139,
          5758,   119,   146,  1519,  1103,  8460,  4958,  1113,  1147,  1319,
           119,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

## Model

In [28]:
from transformers import BertModel

class myModel(torch.nn.Module):

    def __init__(self):

        super(myModel, self).__init__()

        self.bert = BertModel.from_pretrained('bert-large-cased')
        self.bert_dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(768, 4)
        

    def forward(self, input_ids, attention_mask, token_type_ids):

        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, return_dict=True)
        logits = output[0]
        out = self.fc(logits)

        return out



## Training

In [29]:
from transformers import AdamW
from tqdm import tqdm

# Set GPU / CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Put model on device
model = myModel().to(device)

optim = AdamW(model.parameters(), lr=1e-6)

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a 

In [30]:
# Pack data into dataloader by batch
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [31]:
training_epoch = 1

In [32]:
loss_fct = CrossEntropyLoss()

In [33]:
def evaluate(valid_loader):
    model.eval()
    running_loss = 0.0

    with torch.no_grad():
        loop = tqdm(valid_loader, leave=True)
        for batch_id, batch in enumerate(loop):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            q_start = batch['q_start'].to(device)
            r_start = batch['r_start'].to(device)
            q_end = batch['q_end'].to(device)
            r_end = batch['r_end'].to(device)

            # model output
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

            q_start_logits, r_start_logits, q_end_logits, r_end_logits = torch.split(outputs, 1, 2)

            q_start_logits = q_start_logits.squeeze(-1).contiguous()
            r_start_logits = r_start_logits.squeeze(-1).contiguous()
            q_end_logits = q_end_logits.squeeze(-1).contiguous()
            r_end_logits = r_end_logits.squeeze(-1).contiguous()

            q_start_loss = loss_fct(q_start_logits, q_start)
            r_start_loss = loss_fct(r_start_logits, r_start)
            q_end_loss = loss_fct(q_end_logits, q_end)
            r_end_loss = loss_fct(r_end_logits, r_end)

            loss = q_start_loss + r_start_loss + q_end_loss + r_end_loss

            running_loss += loss.item()
            if batch_id % 30 == 0 and batch_id != 0:
                print('Validation Epoch {} Batch {} Loss {:.4f}'.format(
                    batch_id + 1, batch_id, running_loss / 30))
                running_loss = 0.0

        print("evaluate loss: ", loss)

In [34]:
for epoch in range(training_epoch):
    model.train()
    running_loss = 0.0

    loop = tqdm(train_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        # reset
        optim.zero_grad()


        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        q_start = batch['q_start'].to(device)
        r_start = batch['r_start'].to(device)
        q_end = batch['q_end'].to(device)
        r_end = batch['r_end'].to(device)


        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        q_start_logits, r_start_logits, q_end_logits, r_end_logits = torch.split(outputs, 1, 2)

        q_start_logits = q_start_logits.squeeze(-1).contiguous()
        r_start_logits = r_start_logits.squeeze(-1).contiguous()
        q_end_logits = q_end_logits.squeeze(-1).contiguous()
        r_end_logits = r_end_logits.squeeze(-1).contiguous()

        q_start_loss = loss_fct(q_start_logits, q_start)
        r_start_loss = loss_fct(r_start_logits, r_start)
        q_end_loss = loss_fct(q_end_logits, q_end)
        r_end_loss = loss_fct(r_end_logits, r_end)



        loss = q_start_loss + r_start_loss + q_end_loss + r_end_loss

        # calculate loss
        loss.backward()
        # update parameters
        optim.step()

        running_loss += loss.item()
        if batch_id % 50 == 0 and batch_id != 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(
                batch_id + 1, batch_id, running_loss / 50))
            running_loss = 0.0

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
    evaluate(valid_loader)

  0%|          | 0/2979 [00:02<?, ?it/s]


RuntimeError: ignored

In [ ]:
torch.save(model.state_dict(), directory + 'aicup_model_large1201')

In [ ]:
# model = myModel().to(device)
# model.load_state_dict(torch.load(directory + 'aicup_model'))

## Predict

In [ ]:
def predict(test_loader):
    predict_pos = []

    model.eval()

    q_sub_output, r_sub_output = [],[]

    loop = tqdm(test_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)

        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        
        q_start_logits, r_start_logits, q_end_logits, r_end_logits = torch.split(outputs, 1, 2)

        q_start_logits = q_start_logits.squeeze(-1).contiguous()
        r_start_logits = r_start_logits.squeeze(-1).contiguous()
        q_end_logits = q_end_logits.squeeze(-1).contiguous()
        r_end_logits = r_end_logits.squeeze(-1).contiguous()

        q_start_prdict = torch.argmax(q_start_logits, 1).cpu().numpy()
        r_start_prdict = torch.argmax(r_start_logits, 1).cpu().numpy()
        q_end_prdict = torch.argmax(q_end_logits, 1).cpu().numpy()
        r_end_prdict = torch.argmax(r_end_logits, 1).cpu().numpy()

        for i in range(len(input_ids)):
            predict_pos.append((q_start_prdict[i].item(), r_start_prdict[i].item(), q_end_prdict[i].item(), r_end_prdict[i].item()))

            q_sub = tokenizer.decode(input_ids[i][q_start_prdict[i]:q_end_prdict[i]+1])
            r_sub = tokenizer.decode(input_ids[i][r_start_prdict[i]:r_end_prdict[i]+1])
            
            q_sub_output.append(q_sub)
            r_sub_output.append(r_sub)
    
    return q_sub_output, r_sub_output, predict_pos



In [ ]:
q_sub_output, r_sub_output, predict_pos = predict(test_loader)

100%|██████████| 373/373 [01:38<00:00,  3.80it/s]


In [ ]:
def get_output_post_fn(test, q_sub_output, r_sub_output):
    q_sub, r_sub = [], []
    for i in range(len(test)):

        q_sub_pred = q_sub_output[i].split()
        r_sub_pred = r_sub_output[i].split()

        if q_sub_pred is None:
            q_sub_pred = []
        q_sub_error_index = q_sub_pred.index('[SEP]') if '[SEP]' in q_sub_pred else -1

        if q_sub_error_index != -1:
            q_sub_pred = q_sub_pred[:q_sub_error_index]

        temp = r_sub_pred.copy()
        if r_sub_pred is None:
            r_sub_pred = []
        else:
            for j in range(len(temp)):
                if temp[j] == '[SEP]':
                    r_sub_pred.remove('[SEP]')
                if temp[j] == '[PAD]':
                    r_sub_pred.remove('[PAD]')

        q_sub.append(' '.join(q_sub_pred))
        r_sub.append(' '.join(r_sub_pred))

    return q_sub, r_sub

In [ ]:
q_sub, r_sub = get_output_post_fn(test, q_sub_output, r_sub_output)

In [ ]:
test['q_sub'] = q_sub
test['r_sub'] = r_sub

In [ ]:
test.head()

,id,q,r,s,q',r',sub_q_true,sub_r_true,sub_both,q_start,r_start,q_end,r_end,q_sub,r_sub
31107,8144,"Third , it is most tiresome to see all the hor...","DISAGREE:Oh , buddy , Atheism has EVERYTHING t...",DISAGREE,Atheism has nothing to do with the nationalism...,Atheism has EVERYTHING to do with that . The m...,1,1,1,127,22,200,136,,Atheism has EVERYTHING to do with that. The mo...
24980,6644,"You have two tables , one with a small test tu...",AGREE:A no-brainer . Zayonara zygote !,AGREE,I would immediately point in the direction of ...,Zayonara zygote !,1,1,1,297,21,384,37,"You have two tables, one with a small test tub...",A no - brainer. Zayonara zygote!
36566,9568,Let 's assume such a asseration is correct . T...,"DISAGREE:Well , that is obsurd and you know it...",DISAGREE,"Under your logic , you must therefore support ...",that is obsurd and you know it yourself,1,1,1,187,16,340,54,Let's assume such a asseration is correct. Tha...,"If you are going to make arguments, at least m..."
10840,2905,I am not saying what the criminals did was rig...,DISAGREE:You do have a problem with this perso...,DISAGREE,I am not saying what the criminals did was rig...,You do have a problem with this person defendi...,1,1,1,0,9,87,100,I am not saying what the criminals did was rig...,You do have a problem with this person defendi...
37479,9810,I do n't think pro-choicers in general care wh...,DISAGREE:Like campaigning for better informati...,DISAGREE,I do n't think pro-choicers in general care wh...,i give up time and resources to help the local...,1,1,1,0,235,80,305,I don't think pro - choicers in general care w...,Like campaigning for better information on and...


## Grading

In [ ]:
def nltk_token_string(sentence):
    # print(sentence)
    tokens = nltk.word_tokenize(sentence)
    for i in range(len(tokens)):
        if len(tokens[i]) == 1:
            tokens[i] = re.sub(r"[!\"#$%&\'()*\+, -.\/:;<=>?@\[\\\]^_`{|}~]", '', tokens[i])
    while '' in tokens:
        tokens.remove('')
    tokens = ' '.join(tokens)
    return tokens

In [ ]:
def lcs(X, Y):
    X_, Y_ = [], []
    
    X_ = nltk_token_string(X)
    Y_ = nltk_token_string(Y)

    m = len(X_)
    n = len(Y_)
 
    # declaring the array for storing the dp values
    L = [[None]*(n + 1) for i in range(m + 1)]
 
    """Following steps build L[m + 1][n + 1] in bottom up fashion
    Note: L[i][j] contains length of LCS of X[0..i-1]
    and Y[0..j-1]"""
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0 :
                L[i][j] = 0
            elif X_[i-1] == Y_[j-1]:
                L[i][j] = L[i-1][j-1]+1
            else:
                L[i][j] = max(L[i-1][j], L[i][j-1])
 
    # L[m][n] contains the length of LCS of X[0..n-1] & Y[0..m-1]
    return L[m][n]


def acc(full, sub):
    common = lcs(full, sub)
    union = len(full) + len(sub) - common
    accuracy = float(common/union)

    return accuracy

In [ ]:
q_acc_sum = 0
r_acc_sum = 0
for i in range(test.shape[0]):
    q_accuracy = acc(test.iloc[i]["q'"], test.iloc[i]['q_sub'])
    r_accuracy = acc(test.iloc[i]["r'"], test.iloc[i]['r_sub'])

    q_acc_sum += q_accuracy
    r_acc_sum += r_accuracy

print("q accuracy: ", q_acc_sum/test.shape[0])
print("r accuracy: ", r_acc_sum/test.shape[0])

q accuracy:  0.5882853605088036
r accuracy:  0.5555944232472311
